In [130]:
import pandas as pd
import joblib

In [131]:
# Load model
model_data = joblib.load("linear_regression_ev_model.pkl")
lr = model_data['model']
training_columns = model_data['columns']

In [132]:
print(f"Model MAE: {model_data['mae']:.2f} km")
print(f"Model R²: {model_data['r2']:.4f}")

Model MAE: 14.93 km
Model R²: 0.9638


In [133]:
# Create new car with exact training columns
new_car = pd.DataFrame(0, index=[0], columns=training_columns, dtype=float)

In [134]:
# Set features
new_car.at[0, 'battery_capacity_kWh'] = 75
new_car.at[0, 'top_speed_kmh'] = 200
new_car.at[0, 'acceleration_0_100_s'] = 7.5
new_car.at[0, 'fast_charging_power_kw_dc'] = 150
new_car.at[0, 'torque_nm'] = 400
new_car.at[0, 'efficiency_wh_per_km'] = 160
new_car.at[0, 'seats'] = 5
new_car.at[0, 'length_mm'] = 4700
new_car.at[0, 'width_mm'] = 1850
new_car.at[0, 'height_mm'] = 1600
new_car.at[0, 'towing_capacity_kg'] = 1000
new_car.at[0, 'drivetrain_RWD'] = 1
new_car.at[0, 'car_body_type_SUV'] = 1
new_car.at[0, 'cargo_volume_l_500'] = 1
new_car.at[0, 'segment_JD - Large'] = 1

In [135]:
# Predict
predicted_range = lr.predict(new_car)[0]
print(f"\n🔋 Predicted EV Range: {predicted_range:.2f} km")


🔋 Predicted EV Range: 391.87 km


In [136]:
def predict_ev_range(
    battery_capacity_kWh=75,
    top_speed_kmh=200,
    acceleration_0_100_s=7.5,
    fast_charging_power_kw_dc=150,
    torque_nm=400,
    efficiency_wh_per_km=160,
    seats=5,
    length_mm=4700,
    width_mm=1850,
    height_mm=1600,
    towing_capacity_kg=1000,
    cargo_volume_l=500,
    drivetrain="RWD",  # Options: "RWD", "FWD", "AWD"
    segment="JD - Large",  # e.g., "C - Medium", "D - Large", "JD - Large"
    car_body_type="SUV"  # e.g., "SUV", "Sedan", "Hatchback"
):
    """
    Predict EV range with custom feature values
    """
    # Create new car with all columns initialized to 0
    new_car = pd.DataFrame(0, index=[0], columns=training_columns, dtype=float)
    
    # Set numeric features
    new_car.at[0, 'battery_capacity_kWh'] = battery_capacity_kWh
    new_car.at[0, 'top_speed_kmh'] = top_speed_kmh
    new_car.at[0, 'acceleration_0_100_s'] = acceleration_0_100_s
    new_car.at[0, 'fast_charging_power_kw_dc'] = fast_charging_power_kw_dc
    new_car.at[0, 'torque_nm'] = torque_nm
    new_car.at[0, 'efficiency_wh_per_km'] = efficiency_wh_per_km
    new_car.at[0, 'seats'] = seats
    new_car.at[0, 'length_mm'] = length_mm
    new_car.at[0, 'width_mm'] = width_mm
    new_car.at[0, 'height_mm'] = height_mm
    new_car.at[0, 'towing_capacity_kg'] = towing_capacity_kg
    
    # Set categorical features
    # Drivetrain
    if f'drivetrain_{drivetrain}' in training_columns:
        new_car.at[0, f'drivetrain_{drivetrain}'] = 1
    
    # Segment
    if f'segment_{segment}' in training_columns:
        new_car.at[0, f'segment_{segment}'] = 1
    
    # Car body type
    if f'car_body_type_{car_body_type}' in training_columns:
        new_car.at[0, f'car_body_type_{car_body_type}'] = 1
    
    # Cargo volume (find closest match)
    cargo_cols = [col for col in training_columns if col.startswith('cargo_volume_l_')]
    if cargo_cols:
        # Extract numeric values from column names
        cargo_values = []
        for col in cargo_cols:
            try:
                val = int(col.replace('cargo_volume_l_', ''))
                cargo_values.append((col, val))
            except:
                pass
        
        # Find closest cargo volume
        if cargo_values:
            closest = min(cargo_values, key=lambda x: abs(x[1] - cargo_volume_l))
            new_car.at[0, closest[0]] = 1
    
    # Predict
    predicted_range = lr.predict(new_car)[0]
    return predicted_range

In [137]:
# Example 1: Small city car
print("=" * 60)
print("EXAMPLE 1: Small City Car (Compact EV)")
print("=" * 60)
range1 = predict_ev_range(
    battery_capacity_kWh=40,
    top_speed_kmh=150,
    acceleration_0_100_s=9.0,
    fast_charging_power_kw_dc=50,
    torque_nm=200,
    efficiency_wh_per_km=140,
    seats=4,
    length_mm=4000,
    width_mm=1700,
    height_mm=1500,
    towing_capacity_kg=0,
    cargo_volume_l=300,
    drivetrain="FWD",
    segment="B - Compact",
    car_body_type="Hatchback"
)
print(f"🔋 Predicted Range: {range1:.2f} km\n")

EXAMPLE 1: Small City Car (Compact EV)
🔋 Predicted Range: 196.90 km



In [138]:
# Example 2: Large luxury SUV
print("=" * 60)
print("EXAMPLE 2: Large Luxury SUV")
print("=" * 60)
range2 = predict_ev_range(
    battery_capacity_kWh=100,
    top_speed_kmh=220,
    acceleration_0_100_s=5.0,
    fast_charging_power_kw_dc=250,
    torque_nm=800,
    efficiency_wh_per_km=200,
    seats=7,
    length_mm=5000,
    width_mm=2000,
    height_mm=1800,
    towing_capacity_kg=2500,
    cargo_volume_l=700,
    drivetrain="RWD",
    segment="F - Luxury",
    car_body_type="SUV"
)
print(f"🔋 Predicted Range: {range2:.2f} km\n")

EXAMPLE 2: Large Luxury SUV
🔋 Predicted Range: 611.89 km



In [139]:
# Example 3: Mid-size sedan
print("=" * 60)
print("EXAMPLE 3: Mid-Size Sedan")
print("=" * 60)
range3 = predict_ev_range(
    battery_capacity_kWh=60,
    top_speed_kmh=180,
    acceleration_0_100_s=7.0,
    fast_charging_power_kw_dc=120,
    torque_nm=350,
    efficiency_wh_per_km=155,
    seats=5,
    length_mm=4600,
    width_mm=1800,
    height_mm=1450,
    towing_capacity_kg=500,
    cargo_volume_l=450,
    drivetrain="RWD",
    segment="D - Large",
    car_body_type="Sedan"
)
print(f"🔋 Predicted Range: {range3:.2f} km\n")

EXAMPLE 3: Mid-Size Sedan
🔋 Predicted Range: 449.75 km



In [140]:
print("=" * 60)
print("CUSTOM PREDICTION:")
print("=" * 60)
custom_range = predict_ev_range(
    battery_capacity_kWh=85,
    efficiency_wh_per_km=170,
    drivetrain="RWD",
    car_body_type="SUV"
)
print(f"🔋 Predicted Range: {custom_range:.2f} km")

CUSTOM PREDICTION:
🔋 Predicted Range: 433.38 km
